# Resume AI Agent Jupyter Notebook

## TEXT HERE

In [7]:
import os
from dotenv import load_dotenv

try:
    load_dotenv()
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"API Authentication Error, please make sure you have setup your .env with the correct GOOGLE_API_KEY: {e}")

✅ Gemini API key setup complete.


In [16]:
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import load_memory, preload_memory

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [9]:
import logging
import os

# Clean up any previous logs
for log_file in ["logger.log", "web.log", "tunnel.log"]:
    if os.path.exists(log_file):
        os.remove(log_file)
        print(f"🧹 Cleaned up {log_file}")

# Configure logging with DEBUG log level.
logging.basicConfig(
    filename="logger.log",
    level=logging.DEBUG,
    format="%(filename)s:%(lineno)s %(levelname)s:%(message)s",
)

print("✅ Logging configured")

✅ Logging configured


In [10]:
# Automatic retries to contact the Gemini API if it fails
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [19]:
INTAKE_INSTRUCTION = (
    "Take the user's raw resume text, do minimal cleaning and extraction (bullet points, skills, structure)."
)

JOB_RESEARCH_INSTRUCTION = (
    "Given the job title/company the user enters, perform a Google search and summarize the real skills/qualifications from the job posting."
)

RESUME_REWRITE_INSTRUCTION = (
    "Read the user's resume and the researched job insights. Produce improved, tailored bullet points and targeted edits based on the job description."
)


# Resume Intake Agent
resume_intake_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="ResumeIntakeAgent",
    instruction=INTAKE_INSTRUCTION,
    tools = [],
)
logging.info("Resume intake agent created")


# Job Research Agent
job_research_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="JobResearchAgent",
    instruction=JOB_RESEARCH_INSTRUCTION,
    tools = [google_search],
)
logging.info("Job Research agent created")


# Resume Agent
resume_rewrite_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="ResumeRewriteAgent",
    instruction=RESUME_REWRITE_INSTRUCTION,
    tools = [],
)
logging.info("Resume Rewriter agent created")


print("✅ Agents created")

✅ Agents created


In [35]:
intake_runner = InMemoryRunner(resume_intake_agent)
job_runner = InMemoryRunner(job_research_agent)
rewrite_runner = InMemoryRunner(resume_rewrite_agent)

async def agent_pipeline(resume_text: str, job_query: str):
    intake_output = await intake_runner.run_debug(resume_text)
    logging.info("Intake output generated and saved")

    job_query_output = await job_runner.run_debug(job_query)
    logging.info("Job query output searched and saved")


    resume_prompt_input = f"""
    You are ResumeRewriteAgent.

    Here is the parsed resume:

    {intake_output}

    Here is the job info for the target role:

    {job_query_output}
    
    Using BOTH, rewrite the resume with tailored bullet points.
    """

    rewrite_output = await rewrite_runner.run_debug(resume_prompt_input)
    logging.info("Resume output generated.")

    return rewrite_output

print("✅ Pipeline created")

✅ Pipeline created


In [36]:
FAKE_RESUME = """
ALEX CARTER
alex.carter@example.com | (555) 291-8743 | San Diego, CA
github.com/alexcarter-dev | linkedin.com/in/alexcarter

EDUCATION
University of California, San Diego
B.S. in Computer Science, Expected June 2026
Relevant Coursework: Data Structures & Algorithms, Machine Learning, Operating Systems, Database Systems, Software Engineering

TECHNICAL SKILLS
Languages: Python, Java, C++, SQL, JavaScript
Tools & Frameworks: PyTorch, TensorFlow, React, Node.js, Flask, Docker, Git
Concepts: Machine Learning, REST APIs, Distributed Systems, Data Pipelines

PROJECTS
SmartTransit (Python, ML)
• Built a real-time bus arrival prediction model using LSTM networks, improving prediction accuracy by 18%.
• Designed a pipeline to clean, merge, and process 2M+ GPS datapoints from a public transit dataset.
• Containerised the entire system using Docker to support scalable deployment.

PixelForge (React, Node.js)
• Developed a full-stack collaborative drawing platform with real-time canvas syncing over WebSockets.
• Implemented role-based access control and persistent project saving with MongoDB.
• Improved frontend load performance by 30% via code splitting and caching.

GPU Weather Simulator (C++, CUDA)
• Accelerated fluid-dynamics-based weather simulation by 12× using CUDA kernels.
• Wrote custom memory-optimised kernels for particle advection and temperature diffusion.

WORK EXPERIENCE
Software Engineering Intern — Horizon Analytics (June 2024 – September 2024)
• Implemented data processing modules for a telemetry monitoring platform ingesting 30K+ events/sec.
• Wrote Python ETL jobs responsible for 15% overall speed improvement.
• Built automated integration tests using pytest and GitHub Actions.

LEADERSHIP & ACTIVITIES
Computer Science Society — Project Team Lead
• Led a 6-student team developing a campus navigation app with indoor routing.
• Managed weekly standups, roadmap planning, and code reviews.

Hackathons
• Winner (1st Place), SD Hacks 2024 — built a wildfire-risk prediction dashboard using satellite imagery.

ADDITIONAL
• Strong interests in ML engineering, distributed systems, and large-scale backend systems.
• Fluent in English and conversational in Spanish.
"""

#resume_text = input("Paste your resume: ")
resume_text = FAKE_RESUME
job_query = input("Enter a job (e.g., 'Tesla SWE Intern 2026'): ")

result = await agent_pipeline(resume_text, job_query)
print(result)

Enter a job (e.g., 'Tesla SWE Intern 2026'):  Tesla SWE Intern 2026



 ### Created new session: debug_session_id

User > 
ALEX CARTER
alex.carter@example.com | (555) 291-8743 | San Diego, CA
github.com/alexcarter-dev | linkedin.com/in/alexcarter

EDUCATION
University of California, San Diego
B.S. in Computer Science, Expected June 2026
Relevant Coursework: Data Structures & Algorithms, Machine Learning, Operating Systems, Database Systems, Software Engineering

TECHNICAL SKILLS
Languages: Python, Java, C++, SQL, JavaScript
Tools & Frameworks: PyTorch, TensorFlow, React, Node.js, Flask, Docker, Git
Concepts: Machine Learning, REST APIs, Distributed Systems, Data Pipelines

PROJECTS
SmartTransit (Python, ML)
• Built a real-time bus arrival prediction model using LSTM networks, improving prediction accuracy by 18%.
• Designed a pipeline to clean, merge, and process 2M+ GPS datapoints from a public transit dataset.
• Containerised the entire system using Docker to support scalable deployment.

PixelForge (React, Node.js)
• Developed a full-stack collaborati